# Spark Session and Config

In [22]:
from pyspark.sql import SparkSession
from dotenv import load_dotenv
import os

spark = SparkSession.builder \
    .appName("Localspark") \
    .master("local[*]") \
    .config("spark.jars.packages",
        "com.microsoft.sqlserver:mssql-jdbc:12.6.1.jre11") \
    .getOrCreate()
spark.sparkContext.setLogLevel("ERROR")

# Hello World

In [3]:

df = spark.createDataFrame([("hello",), ("world",)], ["word"])
df.show()

+-----+
| word|
+-----+
|hello|
|world|
+-----+



# Loading env file that contain connection string

In [14]:
from dotenv import load_dotenv
import os

# Load variables from .env into os.environ
load_dotenv("env")

True

In [18]:
server_name = os.environ.get("SERVERNAME")
database_name = os.environ.get("DATABASENAME")
username = os.environ.get("USERNAME")
password = os.environ.get("PASSWORD")

# Standard JDBC URL format for SQL Server
jdbc_url = f"jdbc:sqlserver://{server_name}:1433;databaseName={database_name}"

In [16]:
username

'user_cp4d_workshop'

In [17]:
server_name

'cp4d-workshop.database.windows.net'

In [19]:
#table name to read
table_name = "dbo.AP18_CUSTOMER_MASTER"

# Read from our Azure Database

In [20]:
try:
    jdbcDF = spark.read \
        .format("jdbc") \
        .option("driver", "com.microsoft.sqlserver.jdbc.SQLServerDriver") \
        .option("url", jdbc_url) \
        .option("dbtable", table_name) \
        .option("user", username) \
        .option("password", password) \
        .load()

    # Display schema and a few rows if successful
    jdbcDF.printSchema()
    jdbcDF.show(5)

except Exception as e:
    print("JDBC read failed!")
    print("Error:", e)


root
 |-- CUST_ID: string (nullable = true)
 |-- FIRST_NAME: string (nullable = true)
 |-- LAST_NAME: string (nullable = true)
 |-- ADDR1: string (nullable = true)
 |-- ADDR2: string (nullable = true)
 |-- CITY: string (nullable = true)
 |-- STATE: string (nullable = true)
 |-- POSTAL_CODE: string (nullable = true)
 |-- PHONE: string (nullable = true)
 |-- EMAIL: string (nullable = true)

+-------+----------+---------+--------------------+-------+----------+-----+-----------+----------+--------------------+
|CUST_ID|FIRST_NAME|LAST_NAME|               ADDR1|  ADDR2|      CITY|STATE|POSTAL_CODE|     PHONE|               EMAIL|
+-------+----------+---------+--------------------+-------+----------+-----+-----------+----------+--------------------+
|      1|      Neil|Armstrong|        100 Maple St|   NULL|Cincinatti|   OH| 45634-3478|4582347809|apollo11commander...|
|     10|     Frank|   Borman|       3856 King St.|   NULL|  Honolulu|   HW| 99345-0000|5672349065| apollo8man@nasa.gov|
|  

In [12]:
spark.stop()